In [1]:
import json
import csv
import numpy as np
from pprint import pprint

In [2]:
filenames = [
    'Aof-t20 (1024Hz)',
    'Aof-t21 (1024Hz)',
    'Aof-t22 (1024Hz)',
    'Aof-t23 (1024Hz)',
    'Aof-t24 (1024Hz)',
    'Aof-t25 (1024Hz)',
    'Beam-t1 (1024Hz)',
    'Beam-t2 (1024Hz)',
    'Beam-t3 (1024Hz)',
    'Beam-t6 (1024Hz)',
    'Beam-t7 (1024Hz)',
    'Beam-t8 (1024Hz)',
#     'Donut-t1 (1024Hz)',
#     'Donut-t2 (1024Hz)',
#     'Donut-t3 (1024Hz)',
#     'Donut-t4 (1024Hz)',
#     'Donut-t5 (1024Hz)',
#     'Donut-t6 (1024Hz)',
#     'Donut-t7 (1024Hz)',
#     'Eye-t1 (1024Hz)',
    'Eye-t2 (1024Hz)',
    'Eye-t3 (1024Hz)',
    'Eye-t4 (1024Hz)',
    'Eye-t5 (1024Hz)',
    'Eye-t6 (1024Hz)',
    'Eye-t7 (1024Hz)',
    'Faai-t1 (1024Hz)',
    'Faai-t2 (1024Hz)',
    'Faai-t3 (1024Hz)',
    'Faai-t4 (1024Hz)',
    'Faai-t5 (1024Hz)',
    'Faai-t6 (1024Hz)',
    'Fluke-t20 (1024Hz)',
    'Fluke-t21 (1024Hz)',
    'Fluke-t22 (1024Hz)',
    'Fluke-t23 (1024Hz)',
    'Fluke-t24 (1024Hz)',
    'Fluke-t25 (1024Hz)',
    'Fong-t1 (1024Hz)',
    'Fong-t2 (1024Hz)',
    'Fong-t3 (1024Hz)',
    'Fong-t4 (1024Hz)',
    'Fong-t5 (1024Hz)',
    'Fong-t6 (1024Hz)',
    'Fong-t7 (1024Hz)',
    'Fong-t8 (1024Hz)',
    'Joke-t1 (1024Hz)',
    'Joke-t2 (1024Hz)',
    'Joke-t3 (1024Hz)',
    'Joke-t4 (1024Hz)',
    'Joke-t5 (1024Hz)',
    'Joke-t6 (1024Hz)',
    'Pod-t20 (1024Hz)',
    'Pod-t21 (1024Hz)',
    'Pod-t22 (1024Hz)',
    'Pod-t23 (1024Hz)',
    'Pod-t24 (1024Hz)',
    'Pod-t25 (1024Hz)',
    'Pod-t26 (1024Hz)',
    'Tau-t11 (1024Hz)',
    'Tau-t13 (1024Hz)',
    'Tau-t14 (1024Hz)',
    'Tau-t15 (1024Hz)',
    'Tau-t16 (1024Hz)',
    'Tau-t17 (1024Hz)',
    'Tau-t18 (1024Hz)',
    'Toey-t1 (1024Hz)',
    'Toey-t2 (1024Hz)',
    'Toey-t3 (1024Hz)',
    'Toey-t4 (1024Hz)',
    'Toey-t5 (1024Hz)',
    'Toey-t6 (1024Hz)',
    'Toey-t7 (1024Hz)',
    'Tong-t1 (1024Hz)',
    'Tong-t2 (1024Hz)',
    'Tong-t3 (1024Hz)',
    'Tong-t4 (1024Hz)',
    'Tong-t5 (1024Hz)',
    'Tong-t6 (1024Hz)',
#     'Torn-t1 (1024Hz)',
#     'Torn-t2 (1024Hz)',
#     'Torn-t3 (1024Hz)',
    'Torn-t4 (1024Hz)',
    'Torn-t5 (1024Hz)',
    'Torn-t6 (1024Hz)',
    'Torn-t7 (1024Hz)',
    'Torn-t8 (1024Hz)',
    'Torn-t9 (1024Hz)'
]

In [3]:
first_dir = './data/label/1/'
second_dir = './data/label/2/'
out_dir = './data/stat/'

infiles_1 = [first_dir + s + '.json' for s in filenames]
infiles_2 = [second_dir + s + '_LABEL.json' for s in filenames]
outfiles = [out_dir + s + '_STAT.txt' for s in filenames]
outfiles_csv = [out_dir + s + '_STAT.csv' for s in filenames]

In [4]:
def remove_single_segment(label_data_1, label_data_2):
    i = 0
    j = 0
    remove_index_1 = []
    remove_index_2 = []
    
    while i < label_data_1.shape[0] and j < label_data_2.shape[0]:
        if label_data_2[j][0] > label_data_1[i][0] - 2048 and label_data_2[j][0] < label_data_1[i][0] + 2048:
            i += 1
            j += 1
        else:
            if label_data_1[i][0] < label_data_2[j][0]:
                # remove i
                remove_index_1.append(i)
                i += 1
            else:
                # remove j
                remove_index_2.append(j)
                j += 1
    while i < label_data_1.shape[0]:
        remove_index_1.append(i)
        i += 1
    while j < label_data_2.shape[0]:
        remove_index_2.append(j)
        j += 1
    
    mask_1 = np.ones(label_data_1.shape[0], dtype = bool)
    mask_1[remove_index_1] = False
    
    mask_2 = np.ones(label_data_2.shape[0], dtype = bool)
    mask_2[remove_index_2] = False

    return label_data_1[mask_1], label_data_2[mask_2] 

In [5]:
with open('./data/stat_csv/width_by_file_aof.csv', 'w', newline='') as fout_1, open('./data/stat_csv/width_by_file_tong.csv', 'w', newline='') as fout_2, open('./data/stat_csv/starting_point_diff_by_file.csv', 'w', newline='') as fout_3, open('./data/stat_csv/ending_point_diff_by_file.csv', 'w', newline='') as fout_4:
    fieldnames = ['File Name', 'N', 'Mean', 'Med', 'Std', 'Min', 'Max']
    writer_1 = csv.DictWriter(fout_1, fieldnames = fieldnames)
    writer_1.writeheader()
    
    writer_2 = csv.DictWriter(fout_2, fieldnames = fieldnames)
    writer_2.writeheader()
    
    writer_3 = csv.DictWriter(fout_3, fieldnames = fieldnames)
    writer_3.writeheader()
    
    writer_4 = csv.DictWriter(fout_4, fieldnames = fieldnames)
    writer_4.writeheader()
    for i in range(0, len(filenames)):
        print("Processing " + filenames[i])
        with open(infiles_1[i]) as fin_1, open(infiles_2[i]) as fin_2:
            label_data_1 = json.load(fin_1)
            label_data_2 = json.load(fin_2)

            # Convert to numpy array
            np_label_data_1 = np.array(label_data_1)
            np_label_data_2 = np.array(label_data_2)
            
            np_label_data_1, np_label_data_2 = remove_single_segment(np_label_data_1, np_label_data_2)
            
            if len(np_label_data_1) != len(np_label_data_2):
                print("{0}: Length not equal".format(filenames[i]))
                continue

            # Extract starting and ending points
            start_1 = np.array([d['start'] for d in np_label_data_1[:,1]])
            start_2 = np.array([d['start'] for d in np_label_data_2[:,1]])
            end_1 = np.array([d['end'] for d in np_label_data_1[:,1]])
            end_2 = np.array([d['end'] for d in np_label_data_2[:,1]])

            # Calculate width
            width_1 = end_1 - start_1
            width_2 = end_2 - start_2

            # Calculate diff
            start_diff = np.abs(start_1 - start_2)
            end_diff = np.abs(end_1 - end_2)

            stat = {}

            stat['1'] = {}
            stat['1']['mean'] = np.mean(width_1)
            stat['1']['med'] = np.median(width_1)
            stat['1']['min'] = np.min(width_1)
            stat['1']['max'] = np.max(width_1)
            stat['1']['std'] = np.std(width_1)

            writer_1.writerow({'File Name': filenames[i],
                               'N': len(np_label_data_1),
                               'Mean': '{:.6f}'.format(stat['1']['mean']),
                               'Med': '{:.6f}'.format(stat['1']['med']),
                               'Std': '{:.6f}'.format(stat['1']['std']),
                               'Min': '{:.6f}'.format(stat['1']['min']),
                               'Max': '{:.6f}'.format(stat['1']['max'])
                              })

            stat['2'] = {}
            stat['2']['mean'] = np.mean(width_2)
            stat['2']['med'] = np.median(width_2)
            stat['2']['min'] = np.min(width_2)
            stat['2']['max'] = np.max(width_2)
            stat['2']['std'] = np.std(width_2)
            
            writer_2.writerow({'File Name': filenames[i],
                               'N': len(np_label_data_1),
                               'Mean': '{:.6f}'.format(stat['2']['mean']),
                               'Med': '{:.6f}'.format(stat['2']['med']),
                               'Std': '{:.6f}'.format(stat['2']['std']),
                               'Min': '{:.6f}'.format(stat['2']['min']),
                               'Max': '{:.6f}'.format(stat['2']['max'])
                              })

            stat['start_diff'] = {}
            stat['start_diff']['mean'] = np.mean(start_diff)
            stat['start_diff']['med'] = np.median(start_diff)
            stat['start_diff']['min'] = np.min(start_diff)
            stat['start_diff']['max'] = np.max(start_diff)
            stat['start_diff']['std'] = np.std(start_diff)
            
            writer_3.writerow({'File Name': filenames[i],
                               'N': len(np_label_data_1),
                               'Mean': '{:.6f}'.format(stat['start_diff']['mean']),
                               'Med': '{:.6f}'.format(stat['start_diff']['med']),
                               'Std': '{:.6f}'.format(stat['start_diff']['std']),
                               'Min': '{:.6f}'.format(stat['start_diff']['min']),
                               'Max': '{:.6f}'.format(stat['start_diff']['max'])
                              })

            stat['end_diff'] = {}
            stat['end_diff']['mean'] = np.mean(end_diff)
            stat['end_diff']['med'] = np.median(end_diff)
            stat['end_diff']['min'] = np.min(end_diff)
            stat['end_diff']['max'] = np.max(end_diff)
            stat['end_diff']['std'] = np.std(end_diff)
            
            writer_4.writerow({'File Name': filenames[i],
                               'N': len(np_label_data_1),
                               'Mean': '{:.6f}'.format(stat['end_diff']['mean']),
                               'Med': '{:.6f}'.format(stat['end_diff']['med']),
                               'Std': '{:.6f}'.format(stat['end_diff']['std']),
                               'Min': '{:.6f}'.format(stat['end_diff']['min']),
                               'Max': '{:.6f}'.format(stat['end_diff']['max'])
                              })

            fin_1.close()
            fin_2.close()
    fout_1.close()
    fout_2.close()
    fout_3.close()
    fout_4.close()
print("Done.")

Processing Aof-t20 (1024Hz)
Processing Aof-t21 (1024Hz)
Processing Aof-t22 (1024Hz)
Processing Aof-t23 (1024Hz)
Processing Aof-t24 (1024Hz)
Processing Aof-t25 (1024Hz)
Processing Beam-t1 (1024Hz)
Processing Beam-t2 (1024Hz)
Processing Beam-t3 (1024Hz)
Processing Beam-t6 (1024Hz)
Processing Beam-t7 (1024Hz)
Processing Beam-t8 (1024Hz)
Processing Eye-t2 (1024Hz)
Processing Eye-t3 (1024Hz)
Processing Eye-t4 (1024Hz)
Processing Eye-t5 (1024Hz)
Processing Eye-t6 (1024Hz)
Processing Eye-t7 (1024Hz)
Processing Faai-t1 (1024Hz)
Processing Faai-t2 (1024Hz)
Processing Faai-t3 (1024Hz)
Processing Faai-t4 (1024Hz)
Processing Faai-t5 (1024Hz)
Processing Faai-t6 (1024Hz)
Processing Fluke-t20 (1024Hz)
Processing Fluke-t21 (1024Hz)
Processing Fluke-t22 (1024Hz)
Processing Fluke-t23 (1024Hz)
Processing Fluke-t24 (1024Hz)
Processing Fluke-t25 (1024Hz)
Processing Fong-t1 (1024Hz)
Processing Fong-t2 (1024Hz)
Processing Fong-t3 (1024Hz)
Processing Fong-t4 (1024Hz)
Processing Fong-t5 (1024Hz)
Processing Fon